In [78]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
import os
import cv2
import tensorflow as tf
from keras.models import Model
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [29]:
batch_size = 16
num_class = 5
epochs = 10
Freeze_layer = 2

In [3]:
dir_name = '2021-ml-100-marathon-final-exam/image_data/train/daisy'

In [4]:
array_of_img = [] 
target = []
for filename in os.listdir('./'+dir_name):
    img = cv2.imread(dir_name+'/'+filename)
    img = cv2.resize(img,(250,250))
    array_of_img.append(img)
    target.append(0)

In [5]:
dir_name = '2021-ml-100-marathon-final-exam/image_data/train/dandelion'

In [6]:
for filename in os.listdir('./'+dir_name):
    img = cv2.imread(dir_name+'/'+filename)
    img = cv2.resize(img,(250,250))
    array_of_img.append(img)
    target.append(1)

In [7]:
dir_name = '2021-ml-100-marathon-final-exam/image_data/train/rose'

In [8]:
for filename in os.listdir('./'+dir_name):
    img = cv2.imread(dir_name+'/'+filename)
    img = cv2.resize(img,(250,250))
    array_of_img.append(img)
    target.append(2)

In [9]:
dir_name = '2021-ml-100-marathon-final-exam/image_data/train/sunflower'

In [10]:
for filename in os.listdir('./'+dir_name):
    img = cv2.imread(dir_name+'/'+filename)
    img = cv2.resize(img,(250,250))
    array_of_img.append(img)
    target.append(3)

In [11]:
dir_name = '2021-ml-100-marathon-final-exam/image_data/train/tulip'

In [12]:
for filename in os.listdir('./'+dir_name):
    img = cv2.imread(dir_name+'/'+filename)
    img = cv2.resize(img,(250,250))
    array_of_img.append(img)
    target.append(4)

In [13]:
from keras.applications.resnet50 import ResNet50

In [14]:
resnet_model = ResNet50(input_shape=(250,250,3),
                       weights='imagenet',
                       include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\ian\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [15]:
last_featuremaps = resnet_model.output
flatten_featuremap = Flatten()(last_featuremaps)
output = Dense(num_class, activation='softmax')(flatten_featuremap)
New_resnet_model = Model(inputs=resnet_model.input, outputs=output)

In [16]:
for layer in New_resnet_model.layers[:Freeze_layer]:
    layer.trainable = False
for layer in New_resnet_model.layers[Freeze_layer:]:
    layer.trainable = True

In [17]:
New_resnet_model.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy',metrics=['acc'])

In [18]:
New_resnet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250, 250, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 125, 125, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 125, 125, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [19]:
array_of_img = np.array(array_of_img)

In [20]:
array_of_img.shape

(2823, 250, 250, 3)

In [21]:
target = np.array(target)

In [22]:
target = keras.utils.to_categorical(target, num_class)

In [23]:
target.shape

(2823, 5)

In [25]:
augment_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [30]:
history = New_resnet_model.fit_generator(augment_generator.flow(array_of_img, target, batch_size=batch_size),
                                         steps_per_epoch = int(len(array_of_img)/batch_size),
                                         epochs=epochs,
                                         verbose=1)

Epoch 1/10
176/176 [==============================] - 5079s 29s/step - loss: 1.2646 - acc: 0.5885
Epoch 2/10
176/176 [==============================] - 28914s 164s/step - loss: 0.5461 - acc: 0.8147
Epoch 3/10
176/176 [==============================] - 5629s 32s/step - loss: 0.3998 - acc: 0.8607
Epoch 4/10
176/176 [==============================] - 4716s 27s/step - loss: 0.2604 - acc: 0.9038
Epoch 5/10


KeyboardInterrupt: 

In [32]:
New_resnet_model.save('my_model.h5')

In [33]:
model = keras.models.load_model('my_model.h5')

In [34]:
score = model.evaluate(array_of_img , target)

2823/2823 [==============================] - 1222s 433ms/step


In [35]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.10632688425457566
Test accuracy: 0.9656394124031067


In [44]:
test_data = []
test_name = []

In [36]:
dir_name = '2021-ml-100-marathon-final-exam/image_data/test'

In [45]:
for filename in os.listdir('./'+dir_name):
    img = cv2.imread(dir_name+'/'+filename)
    #print(filename[:-4])
    img = cv2.resize(img,(250,250))
    test_name.append(filename[:-4])
    test_data.append(img)

In [47]:
test_data = np.array(test_data)

In [48]:
test_data.shape

(2000, 250, 250, 3)

In [65]:
y = model.predict(test_data)

In [67]:
tmp = y

In [73]:
np.argmax(tmp[0])

2

In [75]:
pred = []
for i in range(len(tmp)):
    pred.append(np.argmax(tmp[i]))

AttributeError: 'list' object has no attribute 'shape'

In [79]:
sub = pd.DataFrame({'id':test_name,'flower_class':pred})

In [83]:
sub.to_csv('test.csv',index=False)

In [84]:
sub

,id,flower_class
0,0028624c49b3e0610ff9f1d111f5d532,2
1,002c30700185b7971369258b438070d5,4
2,00852f4f666acecd0c0d140365b42efd,2
3,00c08828fce04e360c732cac01edad9e,4
4,00d366e7877b6a78b104b57d67b60e6b,4
...,...,...
1995,ff7eac29b6d7a33fbd8009677c3e9c58,1
1996,ffbc32a7b67dfe72b8d35d4b1b35fd6c,1
1997,ffea1f275c05accb0a6bfd1203620c7e,0
1998,ffeb2a1cf53464b6af937ab8af0c2946,4


In [85]:
model.fit_generator(augment_generator.flow(array_of_img, target, batch_size=batch_size),
                                         steps_per_epoch = int(len(array_of_img)/batch_size),
                                         epochs=1,
                                         verbose=1)

Epoch 1/1
120/176 [===================>..........] - ETA: 30:40 - loss: 0.2369 - acc: 0.9240

KeyboardInterrupt: 